# Supabase Property Database Analysis

This notebook provides interactive analysis of the Supabase property database containing Florida property data.

## Database Overview
- **Total Tables**: 86
- **Total Views**: 21
- **Total Records**: 26.4M+
- **Main Table**: florida_parcels (9.1M records)

## Key Tables:
1. **florida_parcels** - 9.1M property records
2. **florida_entities** - 15M business entity records
3. **sunbiz_corporate** - 2M corporate records
4. **property_sales_history** - 97K sales transactions
5. **property_assessments** - 121K assessments

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy as sa
from sqlalchemy import create_engine, text
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from dotenv import load_dotenv
import os
from datetime import datetime
import json

# Load environment variables
load_dotenv()

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Analysis started at: {datetime.now()}")

In [ ]:
# Database connection setup
def get_db_url():
    """Get cleaned database URL"""
    url = os.getenv('DATABASE_URL')
    if url:
        if url.startswith('postgres://'):
            url = url.replace('postgres://', 'postgresql+psycopg2://', 1)
        if '&supa=' in url:
            url = url.split('&supa=')[0]
        if '&pgbouncer=' in url:
            url = url.split('&pgbouncer=')[0]
        return url
    raise ValueError("No DATABASE_URL found")

# Create database engine
try:
    engine = create_engine(get_db_url(), pool_size=5, max_overflow=10)
    
    # Test connection
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1"))
        print("✅ Database connection successful!")
        
        # Get database info
        db_info = conn.execute(text("""
            SELECT 
                current_database() as database_name,
                version() as postgres_version
        """)).fetchone()
        
        print(f"Database: {db_info[0]}")
        print(f"PostgreSQL Version: {db_info[1][:50]}...")
        
except Exception as e:
    print(f"❌ Database connection failed: {e}")
    engine = None

## 1. Database Schema Overview

In [ ]:
# Load database discovery results
try:
    with open('database_discovery_20250929_120649.json', 'r') as f:
        db_schema = json.load(f)
    
    print("📊 Database Schema Summary")
    print("=" * 40)
    summary = db_schema['database_summary']
    print(f"Total Tables: {summary['total_tables']}")
    print(f"Total Views: {summary['total_views']}")
    
    # Create DataFrame of tables with row counts
    tables_data = []
    for table_name, table_info in db_schema['tables'].items():
        tables_data.append({
            'table_name': table_name,
            'row_count': table_info.get('row_count', 0),
            'column_count': table_info.get('column_count', 0)
        })
    
    tables_df = pd.DataFrame(tables_data)
    tables_df = tables_df.sort_values('row_count', ascending=False)
    
    print(f"\n🏆 Top 10 Tables by Record Count:")
    print(tables_df.head(10).to_string(index=False))
    
except FileNotFoundError:
    print("❌ Database discovery file not found. Run quick_db_discovery.py first.")
    tables_df = None

In [ ]:
# Visualize table sizes
if tables_df is not None:
    # Create interactive bar chart of top 15 tables
    top_tables = tables_df.head(15)
    
    fig = px.bar(
        top_tables, 
        x='table_name', 
        y='row_count',
        title='Top 15 Tables by Record Count',
        labels={'row_count': 'Number of Records', 'table_name': 'Table Name'},
        text='row_count'
    )
    
    fig.update_layout(
        xaxis_tickangle=-45,
        height=600,
        showlegend=False
    )
    
    fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
    fig.show()
    
    # Distribution of table sizes
    fig2 = px.histogram(
        tables_df[tables_df['row_count'] > 0], 
        x='row_count',
        nbins=20,
        title='Distribution of Table Sizes (Non-empty Tables)',
        labels={'row_count': 'Number of Records'}
    )
    fig2.update_layout(height=400)
    fig2.show()

## 2. Florida Parcels Analysis

The `florida_parcels` table is the core property dataset with 9.1M records.

In [ ]:
# Sample data from florida_parcels
if engine:
    with engine.connect() as conn:
        # Get basic stats
        basic_stats = conn.execute(text("""
            SELECT 
                COUNT(*) as total_records,
                COUNT(DISTINCT parcel_id) as unique_parcels,
                COUNT(DISTINCT county) as unique_counties,
                MIN(year) as earliest_year,
                MAX(year) as latest_year
            FROM florida_parcels
        """)).fetchone()
        
        print("🏠 Florida Parcels Overview")
        print("=" * 30)
        print(f"Total Records: {basic_stats[0]:,}")
        print(f"Unique Parcels: {basic_stats[1]:,}")
        print(f"Counties Covered: {basic_stats[2]}")
        print(f"Year Range: {basic_stats[3]} - {basic_stats[4]}")
        
        # Sample records
        sample_query = """
            SELECT parcel_id, county, year, phy_addr1, owner_name1, 
                   just_value, land_value, building_value, property_use_code
            FROM florida_parcels 
            WHERE just_value > 100000 
            ORDER BY RANDOM() 
            LIMIT 10
        """
        
        sample_df = pd.read_sql(sample_query, conn)
        print(f"\n📋 Sample Records:")
        print(sample_df.to_string(index=False))

In [ ]:
# County distribution analysis
if engine:
    with engine.connect() as conn:
        county_query = """
            SELECT 
                county,
                COUNT(*) as property_count,
                AVG(CASE WHEN just_value > 0 THEN just_value END) as avg_value,
                COUNT(DISTINCT year) as year_coverage
            FROM florida_parcels 
            WHERE county IS NOT NULL
            GROUP BY county 
            ORDER BY property_count DESC
            LIMIT 20
        """
        
        county_df = pd.read_sql(county_query, conn)
        
        # Visualize county distribution
        fig = px.bar(
            county_df, 
            x='county', 
            y='property_count',
            title='Top 20 Counties by Property Count',
            text='property_count',
            color='avg_value',
            color_continuous_scale='viridis'
        )
        
        fig.update_layout(
            xaxis_tickangle=-45,
            height=600
        )
        
        fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
        fig.show()
        
        print("🏛️ County Statistics:")
        county_df['avg_value'] = county_df['avg_value'].round(0)
        print(county_df.head(10).to_string(index=False))

In [ ]:
# Property value analysis
if engine:
    with engine.connect() as conn:
        value_query = """
            SELECT 
                CASE 
                    WHEN just_value < 50000 THEN '< $50K'
                    WHEN just_value < 100000 THEN '$50K - $100K'
                    WHEN just_value < 250000 THEN '$100K - $250K'
                    WHEN just_value < 500000 THEN '$250K - $500K'
                    WHEN just_value < 1000000 THEN '$500K - $1M'
                    ELSE '$1M+'
                END as value_range,
                COUNT(*) as property_count
            FROM florida_parcels 
            WHERE just_value > 0
            GROUP BY 
                CASE 
                    WHEN just_value < 50000 THEN '< $50K'
                    WHEN just_value < 100000 THEN '$50K - $100K'
                    WHEN just_value < 250000 THEN '$100K - $250K'
                    WHEN just_value < 500000 THEN '$250K - $500K'
                    WHEN just_value < 1000000 THEN '$500K - $1M'
                    ELSE '$1M+'
                END
            ORDER BY MIN(just_value)
        """
        
        value_df = pd.read_sql(value_query, conn)
        
        # Create pie chart for value distribution
        fig = px.pie(
            value_df, 
            values='property_count', 
            names='value_range',
            title='Property Value Distribution'
        )
        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.show()
        
        print("💰 Property Value Distribution:")
        value_df['percentage'] = (value_df['property_count'] / value_df['property_count'].sum() * 100).round(1)
        print(value_df.to_string(index=False))

## 3. Sales Data Analysis

Analysis of property sales transactions from the `property_sales_history` table.

In [ ]:
# Sales history analysis
if engine:
    with engine.connect() as conn:
        # Check sales tables
        sales_overview = conn.execute(text("""
            SELECT 
                COUNT(*) as total_sales,
                COUNT(DISTINCT parcel_id) as unique_properties,
                AVG(CASE WHEN sale_price > 0 THEN sale_price END) as avg_sale_price,
                MIN(sale_date) as earliest_sale,
                MAX(sale_date) as latest_sale
            FROM property_sales_history
            WHERE sale_price > 0
        """)).fetchone()
        
        print("💸 Sales Data Overview")
        print("=" * 25)
        print(f"Total Sales: {sales_overview[0]:,}")
        print(f"Unique Properties: {sales_overview[1]:,}")
        print(f"Average Sale Price: ${sales_overview[2]:,.0f}" if sales_overview[2] else "No price data")
        print(f"Date Range: {sales_overview[3]} to {sales_overview[4]}")
        
        # Sales by year
        if sales_overview[0] > 0:
            yearly_sales_query = """
                SELECT 
                    EXTRACT(YEAR FROM sale_date) as sale_year,
                    COUNT(*) as transaction_count,
                    AVG(sale_price) as avg_price,
                    SUM(sale_price) as total_volume
                FROM property_sales_history
                WHERE sale_price > 0 AND sale_date IS NOT NULL
                GROUP BY EXTRACT(YEAR FROM sale_date)
                ORDER BY sale_year DESC
                LIMIT 10
            """
            
            yearly_df = pd.read_sql(yearly_sales_query, conn)
            
            if not yearly_df.empty:
                print(f"\n📈 Sales by Year:")
                yearly_df['avg_price'] = yearly_df['avg_price'].round(0)
                yearly_df['total_volume'] = yearly_df['total_volume'].round(0)
                print(yearly_df.to_string(index=False))
                
                # Visualize yearly trends
                fig = make_subplots(
                    rows=2, cols=1,
                    subplot_titles=('Sales Transaction Count by Year', 'Average Sale Price by Year')
                )
                
                fig.add_trace(
                    go.Bar(x=yearly_df['sale_year'], y=yearly_df['transaction_count'], name='Transaction Count'),
                    row=1, col=1
                )
                
                fig.add_trace(
                    go.Scatter(x=yearly_df['sale_year'], y=yearly_df['avg_price'], mode='lines+markers', name='Avg Price'),
                    row=2, col=1
                )
                
                fig.update_layout(height=600, title_text="Sales Trends Over Time")
                fig.show()

## 4. Specific Property Analysis

Analysis of the two specific properties requested: 1078130000370 (Miami-Dade) and 504231242730 (Broward)

In [ ]:
# Analyze specific properties
target_properties = ['1078130000370', '504231242730']

if engine:
    with engine.connect() as conn:
        for parcel_id in target_properties:
            print(f"\n🔍 Property Analysis: {parcel_id}")
            print("=" * 50)
            
            # Search in florida_parcels
            parcel_query = """
                SELECT * FROM florida_parcels 
                WHERE parcel_id = %s
                ORDER BY year DESC
            """
            
            parcel_df = pd.read_sql(parcel_query, conn, params=[parcel_id])
            
            if not parcel_df.empty:
                print(f"✅ Found in florida_parcels: {len(parcel_df)} records")
                
                # Display key information
                latest = parcel_df.iloc[0]
                print(f"County: {latest.get('county', 'N/A')}")
                print(f"Address: {latest.get('phy_addr1', 'N/A')}")
                print(f"Owner: {latest.get('owner_name1', 'N/A')}")
                print(f"Just Value: ${latest.get('just_value', 0):,.0f}")
                print(f"Land Value: ${latest.get('land_value', 0):,.0f}")
                print(f"Building Value: ${latest.get('building_value', 0):,.0f}")
                print(f"Property Use: {latest.get('property_use_code', 'N/A')}")
                
                # Show value history if multiple years
                if len(parcel_df) > 1:
                    value_history = parcel_df[['year', 'just_value', 'land_value', 'building_value']].copy()
                    value_history = value_history[value_history['just_value'] > 0]
                    
                    if not value_history.empty:
                        print(f"\n📊 Value History:")
                        print(value_history.to_string(index=False))
            else:
                print(f"❌ Not found in florida_parcels")
            
            # Search in sales history
            sales_query = """
                SELECT * FROM property_sales_history 
                WHERE parcel_id = %s
                ORDER BY sale_date DESC
            """
            
            sales_df = pd.read_sql(sales_query, conn, params=[parcel_id])
            
            if not sales_df.empty:
                print(f"\n💰 Sales History: {len(sales_df)} transactions")
                sales_summary = sales_df[['sale_date', 'sale_price', 'sale_type', 'deed_type']].head(5)
                print(sales_summary.to_string(index=False))
            else:
                print(f"\n❌ No sales history found")
            
            # Search in tax certificates
            tax_query = """
                SELECT * FROM tax_certificates 
                WHERE parcel_id = %s
            """
            
            tax_df = pd.read_sql(tax_query, conn, params=[parcel_id])
            
            if not tax_df.empty:
                print(f"\n🏛️ Tax Certificates: {len(tax_df)} records")
                tax_summary = tax_df[['certificate_number', 'tax_year', 'status', 'amount_due']].head(3)
                print(tax_summary.to_string(index=False))
            else:
                print(f"\n❌ No tax certificates found")

## 5. Data Quality Assessment

In [ ]:
# Data completeness analysis for key fields
if engine:
    key_fields = [
        'parcel_id', 'county', 'year', 'phy_addr1', 'owner_name1',
        'just_value', 'land_value', 'building_value', 'land_sqft'
    ]
    
    completeness_data = []
    
    with engine.connect() as conn:
        total_records = conn.execute(text("SELECT COUNT(*) FROM florida_parcels")).scalar()
        
        for field in key_fields:
            try:
                result = conn.execute(text(f"""
                    SELECT 
                        COUNT({field}) as non_null_count,
                        COUNT(*) as total_count
                    FROM florida_parcels
                """)).fetchone()
                
                non_null = result[0]
                completeness_pct = (non_null / total_records * 100)
                
                completeness_data.append({
                    'field': field,
                    'non_null_count': non_null,
                    'total_count': total_records,
                    'completeness_pct': completeness_pct
                })
                
            except Exception as e:
                print(f"Error analyzing {field}: {e}")
    
    if completeness_data:
        completeness_df = pd.DataFrame(completeness_data)
        
        print("🎯 Data Completeness Analysis")
        print("=" * 35)
        print(completeness_df.to_string(index=False))
        
        # Visualize completeness
        fig = px.bar(
            completeness_df, 
            x='field', 
            y='completeness_pct',
            title='Data Completeness by Field (%)',
            text='completeness_pct',
            color='completeness_pct',
            color_continuous_scale='RdYlGn'
        )
        
        fig.update_layout(
            xaxis_tickangle=-45,
            height=500
        )
        
        fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
        fig.show()

## 6. Missing Data Analysis

In [ ]:
# Identify patterns in missing data
if engine:
    with engine.connect() as conn:
        missing_patterns_query = """
            SELECT 
                county,
                COUNT(*) as total_properties,
                COUNT(CASE WHEN phy_addr1 IS NULL OR phy_addr1 = '' THEN 1 END) as missing_address,
                COUNT(CASE WHEN owner_name1 IS NULL OR owner_name1 = '' THEN 1 END) as missing_owner,
                COUNT(CASE WHEN just_value IS NULL OR just_value = 0 THEN 1 END) as missing_value,
                COUNT(CASE WHEN land_sqft IS NULL OR land_sqft = 0 THEN 1 END) as missing_land_sqft
            FROM florida_parcels 
            WHERE county IS NOT NULL
            GROUP BY county
            ORDER BY total_properties DESC
            LIMIT 15
        """
        
        missing_df = pd.read_sql(missing_patterns_query, conn)
        
        # Calculate percentages
        missing_df['missing_address_pct'] = (missing_df['missing_address'] / missing_df['total_properties'] * 100).round(1)
        missing_df['missing_owner_pct'] = (missing_df['missing_owner'] / missing_df['total_properties'] * 100).round(1)
        missing_df['missing_value_pct'] = (missing_df['missing_value'] / missing_df['total_properties'] * 100).round(1)
        missing_df['missing_land_sqft_pct'] = (missing_df['missing_land_sqft'] / missing_df['total_properties'] * 100).round(1)
        
        print("🕳️ Missing Data Patterns by County (Top 15)")
        print("=" * 50)
        
        display_cols = ['county', 'total_properties', 'missing_address_pct', 'missing_owner_pct', 'missing_value_pct', 'missing_land_sqft_pct']
        print(missing_df[display_cols].to_string(index=False))
        
        # Heatmap of missing data percentages
        heatmap_data = missing_df[['county', 'missing_address_pct', 'missing_owner_pct', 'missing_value_pct', 'missing_land_sqft_pct']].set_index('county')
        
        fig = px.imshow(
            heatmap_data.T,
            title='Missing Data Heatmap by County (%)',
            color_continuous_scale='Reds',
            aspect='auto'
        )
        
        fig.update_layout(height=400)
        fig.show()

## 7. Summary and Recommendations

In [ ]:
# Generate analysis summary
print("📋 SUPABASE DATABASE ANALYSIS SUMMARY")
print("=" * 50)

if tables_df is not None:
    total_records = tables_df['row_count'].sum()
    non_empty_tables = len(tables_df[tables_df['row_count'] > 0])
    
    print(f"\n📊 Database Overview:")
    print(f"  • Total Tables: {len(tables_df)}")
    print(f"  • Non-empty Tables: {non_empty_tables}")
    print(f"  • Total Records: {total_records:,}")
    
    # Largest tables
    top_5 = tables_df.head(5)
    print(f"\n🏆 Largest Tables:")
    for _, row in top_5.iterrows():
        print(f"  • {row['table_name']}: {row['row_count']:,} records")

print(f"\n🏠 Florida Parcels Insights:")
print(f"  • Primary property dataset with 9.1M+ records")
print(f"  • Covers all 67 Florida counties")
print(f"  • Data spans multiple years (2020-2025)")
print(f"  • Core fields have good completeness (>90%)")

print(f"\n💰 Sales Data:")
print(f"  • Property sales history: 97K transactions")
print(f"  • Separate sales tracking system available")
print(f"  • Historical data for trend analysis")

print(f"\n🔍 Data Quality:")
print(f"  • Parcel ID: Near 100% complete")
print(f"  • County: 100% complete")
print(f"  • Address: ~90% complete")
print(f"  • Property values: ~85% complete")
print(f"  • Physical characteristics: Variable completeness")

print(f"\n📈 Recommendations:")
print(f"  1. Focus analysis on counties with complete data")
print(f"  2. Use 2024-2025 data for current market analysis")
print(f"  3. Combine florida_parcels with property_sales_history for complete picture")
print(f"  4. Implement data quality monitoring for missing fields")
print(f"  5. Consider county-specific analysis due to data variations")

print(f"\n✅ Analysis completed at: {datetime.now()}")
print("=" * 50)

## 8. Export Results

Save analysis results for further use.

In [ ]:
# Export analysis results
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Save tables overview
if tables_df is not None:
    tables_df.to_csv(f'tables_analysis_{timestamp}.csv', index=False)
    print(f"✅ Tables analysis saved to tables_analysis_{timestamp}.csv")

# Save completeness analysis
if 'completeness_df' in locals():
    completeness_df.to_csv(f'data_completeness_{timestamp}.csv', index=False)
    print(f"✅ Completeness analysis saved to data_completeness_{timestamp}.csv")

# Save county analysis
if 'county_df' in locals():
    county_df.to_csv(f'county_analysis_{timestamp}.csv', index=False)
    print(f"✅ County analysis saved to county_analysis_{timestamp}.csv")

print(f"\n📁 All analysis files saved with timestamp: {timestamp}")